In [16]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np

from dataset.market import Market
from dataset.build_market import build_market_dataloader
from net.os_net import get_embedding_model, osnet_ibn_x1_0, OSNet, OSBlock
from net.resnet import *
from net.googlenet import *
from net.bn_inception import *
from dataset import sampler
from eval import inference
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

from tqdm import *
import datetime
import wandb

In [17]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [18]:
train_loader, val_dict, num_classes = build_market_dataloader()

=> Market1501 loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


In [19]:
model_name = 'osnet'

In [28]:
sz_embed = 512
criterion = losses.Proxy_Anchor(nb_classes = num_classes, sz_embed = sz_embed, mrg = 0.1, alpha = 32).cuda()

In [29]:
if model_name == 'resnet50':
    model = Resnet50(embedding_size=sz_embed, pretrained=True, is_norm=1, bn_freeze =1).cuda()

    param_groups = [
        {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters())))},
        {'params': model.model.embedding.parameters(), 'lr':float(1e-4) * 1},
    ]
    param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

    modules = model.model.modules()
    for m in modules: 
        if isinstance(m, nn.BatchNorm2d):
            m.eval()
elif model_name == 'osnet':
    model = osnet_ibn_x1_0(pretrained=True).cuda()
    
    param_groups = [
        {'params': list(set(model.parameters()).difference(set(model.fc.parameters())))},
        {'params': model.fc.parameters(), 'lr':float(1e-4) * 1},
    ]
    param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

    modules = model.modules()
    for m in modules: 
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

Successfully loaded imagenet pretrained weights from "/home/nevin/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [30]:
opt = torch.optim.AdamW(param_groups, lr=float(1e-4), weight_decay = 1e-4)

In [31]:
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma = 0.5)

In [32]:
model_dir = 'models'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [33]:
def append_result(result, model, epoch):
    filename = f"{model_dir}/results_{current_datetime}.txt"

    # Open the file in append mode and write the result
    with open(filename, "w") as file:
        file.write(f'Epoch: {epoch}\n')
        file.write(str(result[0]) + "\n")
        file.write('CMC\n')
        file.write(str(result[0]) + "\n")
        file.write('mAP\n')
        file.write(str(result[1]) + "\n\n")
    torch.save({'model_state_dict':model.state_dict()}, '{}/{}_{}_best.pth'.format(model_dir, model_name, current_datetime))

In [ ]:
best_mAP=0
current_datetime = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
for epoch in range(0, 600):
    model.train()

    losses_per_epoch = []
    if model_name == 'resnet50':
        unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())
    else:
        unfreeze_model_param = list(model.fc.parameters()) + list(criterion.parameters())

    if epoch == 0:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = False
    if epoch == 1:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = True

    pbar = tqdm(enumerate(train_loader))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        
        torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(train_loader),
                100. * batch_idx / len(train_loader),
                loss.item()))
    scheduler.step()
    
    val_loader = val_dict['val_loader']
    num_query = val_dict['num_query']
    result = inference(model, val_loader, num_query)

    if result[1] > best_mAP:
        append_result(result, model, epoch)

Train Epoch: 0 [259/259 (100%)] Loss: 3.273218: : 259it [00:30,  8.53it/s]

Enter inferencing
Create evaluator


The test feature is normalized
Validation Results
mAP: 1.9%
CMC curve, Rank-1  :6.3%
CMC curve, Rank-2  :9.2%
CMC curve, Rank-4  :13.1%
CMC curve, Rank-6  :15.8%
CMC curve, Rank-8  :18.2%
CMC curve, Rank-10 :20.0%
CMC curve, Rank-12 :21.9%
CMC curve, Rank-14 :23.5%
CMC curve, Rank-16 :25.1%
CMC curve, Rank-18 :26.6%
CMC curve, Rank-20 :28.1%
CMC curve, Rank-22 :29.3%
CMC curve, Rank-24 :30.3%
CMC curve, Rank-26 :31.3%
CMC curve, Rank-28 :32.3%
CMC curve, Rank-30 :33.2%
CMC curve, Rank-32 :34.2%


Train Epoch: 1 [259/259 (100%)] Loss: 0.933172: : 259it [01:24,  3.07it/s]

Enter inferencing
Create evaluator


...

The test feature is normalized
Validation Results
mAP: 62.5%
CMC curve, Rank-1  :84.0%
CMC curve, Rank-2  :89.0%
CMC curve, Rank-4  :92.6%
CMC curve, Rank-6  :94.5%
CMC curve, Rank-8  :95.5%
CMC curve, Rank-10 :96.4%
CMC curve, Rank-12 :96.9%
CMC curve, Rank-14 :97.1%
CMC curve, Rank-16 :97.4%
CMC curve, Rank-18 :97.6%
CMC curve, Rank-20 :97.7%
CMC curve, Rank-22 :97.9%
CMC curve, Rank-24 :98.0%
CMC curve, Rank-26 :98.0%
CMC curve, Rank-28 :98.2%
CMC curve, Rank-30 :98.4%
CMC curve, Rank-32 :98.5%


Train Epoch: 181 [259/259 (100%)] Loss: 0.042575: : 259it [01:25,  3.01it/s]

Enter inferencing
Create evaluator


The test feature is normalized
Validation Results
mAP: 62.5%
CMC curve, Rank-1  :83.8%
CMC curve, Rank-2  :89.0%
CMC curve, Rank-4  :92.8%
CMC curve, Rank-6  :94.4%
CMC curve, Rank-8  :95.7%
CMC curve, Rank-10 :96.4%
CMC curve, Rank-12 :96.8%
CMC curve, Rank-14 :97.1%
CMC curve, Rank-16 :97.4%
CMC curve, Rank-18 :97.7%
CMC curve, Rank-20 :97.8%
CMC curve, Rank-22 :97.9%
CMC curve, Rank-24 :98.1%
CMC curve, Rank-26 :98.2%
CMC curve, Rank-28 :98.2%
CMC curve, Rank-30 :98.4%
CMC curve, Rank-32 :98.4%


Train Epoch: 182 [117/259 (45%)] Loss: 0.030060: : 117it [00:38,  2.98it/s]

### Test codes